In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from PIL import Image
import tensorflow as tf
import os
import pandas as pd
import numpy
import random as rand

In [3]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
def randfloat(a, b):
    return rand.random() * (b - a) + a

def my_print(s):
    sys.stdout.write(s)
    sys.stdout.flush()

In [4]:
data_dir = "./data/kingstreet/frames/001/"
save_dir = "./save_ks"
save_file_path = "./save_ks/model.ckpt"
logs_dir = "./logs_ks"
# logs_dir_2 = "./logs2"

create_dir(save_dir)
create_dir(logs_dir)
# create_dir(logs_dir_2)

BATCH_SIZE = 16
IMAGE_SHAPE = [405, 720, 3]

frames_cnt = len(os.listdir(data_dir))

_debug_list = []

In [5]:
def _create_label_producers():
    
    lines = [os.path.join(data_dir, str(i) + ".png") + " " + os.path.join(data_dir, str(i + 1) + ".png") + " 1"
             if rand.randint(0, 1) == 0 else 
             os.path.join(data_dir, str(i + 1) + ".png") + " " + os.path.join(data_dir, str(i) + ".png") + " 0"
             
             for i in xrange(frames_cnt - 1)]
    
    cnt = len(lines)
    p = 0.9
    train_cnt = int(cnt * p)
    print "Total: %i images, train: %i" % (cnt, train_cnt)
    train_lines = lines[:train_cnt]
    test_lines = lines[train_cnt:]
    return tf.train.string_input_producer(train_lines), tf.train.string_input_producer(test_lines)

def _read_image_and_label(path_label_line):
    with tf.name_scope("read_image_and_label"):
        path1, path2, label_str = tf.decode_csv(path_label_line, [[""], [""], [""]], field_delim=" ")
        label = tf.cast(tf.string_to_number(label_str, out_type=tf.int32), tf.int64)
        file_content1 = tf.read_file(path1)
        file_content2 = tf.read_file(path2)
        image1 = tf.image.decode_png(file_content1)
        image2 = tf.image.decode_png(file_content2)
        image1 = tf.cast(image1, tf.float32)
        image2 = tf.cast(image2, tf.float32)
        image1.set_shape(IMAGE_SHAPE)
        image2.set_shape(IMAGE_SHAPE)
    return image1, image2, label

def _generate_batch(image1, image2, label, batch_size, min_after_dequeue=200):
    with tf.name_scope("generate_batch"):
        images1, images2, labels = tf.train.shuffle_batch([image1, image2, label], batch_size=batch_size, 
                                                capacity=min_after_dequeue + batch_size * 4, 
                                                min_after_dequeue=min_after_dequeue,
                                                num_threads=5)
    return images1, images2, tf.reshape(labels, [batch_size])
    
def get_data_batch():
    train_queue, test_queue = _create_label_producers()
    image1_train, image2_train, label_train = _read_image_and_label(train_queue.dequeue())
    image1_test, image2_test, label_test = _read_image_and_label(test_queue.dequeue())
    images1_train, images2_train, labels_train = _generate_batch(image1_train, image2_train, 
                                                                 label_train, BATCH_SIZE)
    images1_test, images2_test, labels_test = _generate_batch(image1_test, image2_test, 
                                                              label_test, BATCH_SIZE,
                                                              min_after_dequeue=BATCH_SIZE)
    return images1_train, images2_train, labels_train, images1_test, images2_test, labels_test

def dense_to_one_hot(label_batch, num_labels=2):
    with tf.name_scope("one_hot_encoder"):
        sparse_labels = tf.cast(tf.reshape(label_batch, [-1, 1]), tf.int32)
        derived_size = tf.shape(sparse_labels)[0]
        indices = tf.reshape(tf.range(0, derived_size, 1), [-1, 1])
        concated = tf.concat(1, [indices, sparse_labels])
        outshape = tf.concat(0, [tf.reshape(derived_size, [1]), tf.reshape(num_labels, [1])])
        labels = tf.sparse_to_dense(concated, outshape, 1.0, 0.0)
        labels = tf.cast(labels, tf.float32)
    return labels

In [6]:
def _activation_summary(x):
    tensor_name = x.op.name
    tf.histogram_summary(tensor_name + '/activations', x)
    tf.scalar_summary(tensor_name + '/sparsity', tf.nn.zero_fraction(x))


def _create_conv(layer, kernel_shape, kernel_stddev=1e-3, activation=tf.nn.relu, name="conv", trainable=True):
    with tf.name_scope(name):
        kernel = tf.Variable(tf.truncated_normal(kernel_shape, stddev=kernel_stddev), name="kernel", 
                             trainable=trainable)
        tf.add_to_collection('main', kernel)
        conv = tf.nn.conv2d(layer, kernel, strides=[1, 1, 1, 1], padding="SAME")
        biases = tf.Variable(tf.constant(0.0, shape=[kernel_shape[3]]), name="biases", trainable=trainable)
        tf.add_to_collection('main', biases)
        biased = tf.nn.bias_add(conv, biases)
        conv_relu = activation(biased)
        _activation_summary(conv_relu)

    return conv_relu
    
def _create_pool(layer, kernel_width, stride=1, name="pool"):
    with tf.name_scope(name):
        pool = tf.nn.max_pool(layer, ksize=[1, kernel_width, kernel_width, 1], strides=[1, stride, stride, 1],
                             padding='SAME')
    return pool

def _create_norm(layer, name="norm"):
    with tf.name_scope(name):
        norm = tf.nn.lrn(layer, alpha=0.001 / 9.0, beta=0.75)
    return norm

def _create_full(layer, result_size, weights_stddev=.04, activation=tf.nn.relu, name="full", trainable=True):
    with tf.name_scope(name):
        layer_size = 1
        for k in layer.get_shape()[1:].as_list():
            layer_size *= k
        reshaped = tf.reshape(layer, [BATCH_SIZE, layer_size])
        weights = tf.Variable(tf.truncated_normal([layer_size, result_size], stddev=weights_stddev), name="weights",
                             trainable=trainable)
        tf.add_to_collection('main', weights)
        biases = tf.Variable(tf.constant(.1, shape=[result_size]), name="biases", trainable=trainable)
        tf.add_to_collection('main', biases)
        if activation:
            biased = tf.add(tf.matmul(reshaped, weights), biases)
            result = activation(biased)
        else:
            result = tf.add(tf.matmul(reshaped, weights), biases)
        
        _activation_summary(result)
    return result
    
def build_classifier(images1, images2, trainable=True):
    images = tf.concat(3, [images1, images2])
    
    conv1 = _create_conv(images, [7, 7, 6, 8], name="conv1", trainable=trainable)
    pool1 = _create_pool(conv1, 3, 2, name="pool1")
    norm1 = _create_norm(pool1, name="norm1")
    
    conv2 = _create_conv(norm1, [5, 5, 8, 16], name="conv2", trainable=trainable)
    pool2 = _create_pool(conv2, 3, 2, name="pool2")
    norm2 = _create_norm(pool2, name="norm2")
    
    conv3 = _create_conv(norm2, [5, 5, 16, 16], name="conv3", trainable=trainable)
    pool3 = _create_pool(conv3, 3, 2, name="pool3")
#     norm3 = _create_norm(pool3, name="norm3")
    
    full1 = _create_full(pool3, 128, name="full1", activation=tf.nn.relu, trainable=trainable)
    full2 = _create_full(full1, 64, name="full2", activation=tf.nn.relu, trainable=trainable)
    full3 = _create_full(full2, 2, name="full3", activation=None, trainable=trainable)
    
    softmax = tf.nn.softmax(full3, name="softmax")
    
    return softmax

In [7]:
def build_loss(logits, labels):
    with tf.name_scope("cross_entropy"):
        labels = tf.cast(labels, tf.int64)
        dense_labels = dense_to_one_hot(labels)
        clipped_logits = tf.clip_by_value(logits, 0.00001, 100.0)
        cross_entropy = -dense_labels * tf.log(clipped_logits)
#         cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, dense_labels, 
#                                                                 name='cross_entropy_per_example')
        cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')

    return cross_entropy_mean

In [8]:
def build_train(loss_op, step, init_rate=0.1, decay_steps=8000):

    with tf.name_scope("train"):
        learning_rate = tf.train.exponential_decay(init_rate, step, decay_steps, 0.1, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        grads = optimizer.compute_gradients(loss_op)

        apply_gradient_op = optimizer.apply_gradients(grads, global_step=step)

        for var in tf.trainable_variables():
            tf.histogram_summary(var.op.name, var)

        for grad, var in grads:
            if grad:
                tf.histogram_summary(var.op.name + '/gradients', grad)

    return apply_gradient_op

In [ ]:
def load(saver, sess):
    saver.restore(sess, save_file_path)
    print "Model restored."
    
def save(saver, sess):
    save_path = saver.save(sess, save_file_path)
    print "Model saved in file: %s" % save_path

def train_classifier(need_load):
    global _debug_list
    _debug_list = []
    N = 50000
    
    with tf.Graph().as_default() as g: 
        
        step = tf.Variable(0, trainable=False, name="step")
        tf.add_to_collection("main", step)
    
        images1_train, images2_train, labels_train, images1_test, images2_test, labels_test = get_data_batch()
        images1_ph = tf.placeholder(tf.float32, [BATCH_SIZE] + IMAGE_SHAPE, "images1")
        images2_ph = tf.placeholder(tf.float32, [BATCH_SIZE] + IMAGE_SHAPE, "images2")
        labels_ph = tf.placeholder(tf.int64, [BATCH_SIZE], "labels")
        answer_op = build_classifier(images1_ph, images2_ph, trainable=True)
        loss_op = build_loss(answer_op, labels_ph)
        train_op = build_train(loss_op, step)
        correct_prediction = tf.equal(tf.argmax(answer_op, 1), labels_ph)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        accuracy_summary = tf.scalar_summary("accuracy", accuracy)

        merged_summaries = tf.merge_all_summaries()

        init = tf.initialize_all_variables()

        sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
        saver = tf.train.Saver(var_list=tf.get_collection("main"))
        coord = tf.train.Coordinator()
        writer = tf.train.SummaryWriter(logs_dir, sess.graph_def, flush_secs=30)

        sess.run(init)
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)

        try:

            if need_load:
                load(saver, sess)

            my_print("Starting...\n")

            for i in xrange(0, N):
                if i % 11 == 0:
                    im1, im2, lab = sess.run([images1_test, images2_test, labels_test])
                    feed = {
                        images1_ph : im1,
                        images2_ph : im2,
                        labels_ph : lab
                    }
                    result = sess.run([merged_summaries, accuracy, step], feed_dict=feed)
                    summary_str = result[0]
                    acc = result[1]
                    st = result[2]
                    writer.add_summary(summary_str, st)
                    print("Accuracy at step %s: %s" % (st, acc))
                else:
                    im1, im2, lab = sess.run([images1_train, images2_train, labels_train])
                    feed = {
                        images1_ph : im1,
                        images2_ph : im2,
                        labels_ph : lab
                    }
                    sess.run(train_op, feed_dict=feed)

                if i % 100 == 0:
                    save(saver, sess)
        finally:
            coord.request_stop()
            coord.join(threads)


In [ ]:
train_classifier(need_load=False)

Total: 128709 images, train: 115838
Starting...
Accuracy at step 0: 0.5625
Model saved in file: ./save_ks/model.ckpt
Accuracy at step 10: 0.3125
Accuracy at step 20: 0.6875
Accuracy at step 30: 0.5
Accuracy at step 40: 0.5
Accuracy at step 50: 0.3125
Accuracy at step 60: 0.25
Accuracy at step 70: 0.5
Accuracy at step 80: 0.5
Accuracy at step 90: 0.625
Model saved in file: ./save_ks/model.ckpt
Accuracy at step 100: 0.5
Accuracy at step 110: 0.375
Accuracy at step 120: 0.375
Accuracy at step 130: 0.3125
Accuracy at step 140: 0.6875
Accuracy at step 150: 0.5
Accuracy at step 160: 0.5
Accuracy at step 170: 0.4375
Accuracy at step 180: 0.625
Model saved in file: ./save_ks/model.ckpt
Accuracy at step 190: 0.5625
Accuracy at step 200: 0.25
Accuracy at step 210: 0.3125
Accuracy at step 220: 0.4375
Accuracy at step 230: 0.5
Accuracy at step 240: 0.6875
Accuracy at step 250: 0.4375
Accuracy at step 260: 0.4375
Accuracy at step 270: 0.5625
Model saved in file: ./save_ks/model.ckpt
Accuracy at ste